<a href="https://colab.research.google.com/github/Jair-RM/Simulacion-2-Figols/blob/main/Ejercicio_05_Nov_(No_se).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Cox Ingersoll Ross Milstrein datos reales BIEN
from scipy.stats import kstest, gaussian_kde, chi2
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from random import random, gauss, seed
from math import log, exp, sqrt, pi
import numpy as np
from time import time
from numba import jit

@jit(nopython=True)
def F(z,u,o,c):
  return c*(z-u)

@jit(nopython=True)
def G(z,u,o,c):
  return o*sqrt(z)

@jit(nopython=True)
def Gp(z,u,o,c):
  return o/(2*sqrt(z))

@jit(nopython=True)
def F1(z,u,o,c,a):
  return c*(z-u) + a*z**2

@jit(nopython=True)
def G1(z,u,o,c):
  return o*sqrt(z)

@jit(nopython=True)
def Gp1(z,u,o,c):
  return o/(2*sqrt(z))

@jit(nopython=True)
def K(x):
  return (1/sqrt(2*pi))*exp(-x**2/2)

def datos(n, u, o, c, delta, desplazamiento, indi, a):
  if indi==1:
    dat=np.loadtxt("tasas1.txt")
    ly=np.zeros(n)
    for i in range(n):
      ly[i]=dat[i + desplazamiento]
  else:
    ly=simula1(u,u,o,c,delta,n,a)
  return ly

@jit(nopython=True)
def pdf(z,h,l):
  m=len(l)
  suma=0
  for i in range(m):
    suma=suma+K((z-l[i])/h)
  return suma/(m*h)

@jit(nopython=True)
def simula(z0,u,o,c,delta,n):
  oo=sqrt(delta)
  ly=np.zeros(n)
  ly[0]=z0
  for i in range(n-1):
    z=ly[i]
    t=0
    while t<1:
      w=oo*gauss(0,1)
      z=z+ F(z,u,o,c) + G(z,u,o,c)*w + (1/2)*G(z,u,o,c)*Gp(z,u,o,c)*(w**2-delta)
      t=t+delta
    ly[i+1]=z
  return ly

@jit(nopython=True)
def simula1(z0, u, o, c, delta, n, a):
  oo=sqrt(delta)
  ly=np.zeros(n)
  ly[0]=z0
  for i in range(n-1):
    z=ly[i]
    t=0
    while t<1:
      w=oo*gauss(0,1)
      z=z+ F1(z,u,o,c,a)*delta + G(z,u,o,c)*w + (1/2)*G(z,u,o,c)*Gp(z,u,o,c)*(w**2 - delta)
      t=t + delta
    ly[i+1]=z
  return ly

@jit(nopython=True)
def f(x, ly, delta, ns, semilla):
  seed(semilla)
  oo=sqrt(delta)
  u=x[0]
  o=x[1]
  c=x[2]
  suma=0
  m=len(ly)
  for i in range(m):
    l=np.zeros(ns)
    for j in range(ns):
      z=ly[i]
      t=0
      while t<1:
        w=oo*gauss(0,1)
        z=z+ F1(z,u,o,c,a)*delta + G(z,u,o,c)*w + (1/2)*G(z,u,o,c)*Gp(z,u,o,c)*(w**2 - delta)
        t=t + delta
      l[j]=z
    o1=np.std(1)
    h=1.06*o1/(ns**0.2)
    p=pdf(ly[i+1], h, l)
    if p<1e-200:
      p=1e-200
    suma=suma+log(p)
  #print("%10.6f"%u, "%10.6f"%o, "%10.6f"%c, "%10.6f"%suma)
  return -suma

@jit(nopython=True)
def prueba(x,ly,delta,ns):
  u=x[0]
  o=x[1]
  c=x[2]
  m=len(ly)
  oo=sqrt(delta)
  l1=np.zeros(m-1)
  for i in range(m-1):
    l1=np.zeros(ns)
    for j in range(ns):
      z=ly[i]
      t=0
      while t<1:
        w=oo*gauss(0,1)
        z=z+ F1(z,u,o,c,a)*delta + G(z,u,o,c)*w + (1/2)*G(z,u,o,c)*Gp(z,u,o,c)*(w**2 - delta)
        t=t + delta
      l1[j]=z
    l1=np.sort(l1)
    j=0
    while j<ns and l1[j]<=ly[i+1]:
      j=j+1
    valor=j/ns
    l1[j]=valor
  return l1

@jit(nopython=True)
def test(x,nr,delta,ly):
  u=x[0]
  o=x[1]
  c=x[2]
  ooo=sqrt(delta)
  M=nr
  N=len(ly)
  sim=np.zeros((N-1, nr))
  for i in range(M):
    for j in range(N-1):
      z=ly[j]
      t=0
      while t<1:
        w=ooo*gauss(0,1)
        z=z+ F1(z,u,o,c,a)*delta + G(z,u,o,c)*w + (1/2)*G(z,u,o,c)*Gp(z,u,o,c)*(w**2 - delta)
        t=t + delta
      sim[j,i]=z
  r=np-zeros(N-1)
  for j in range(N-1):
    suma=1
    for i in range(M):
      if ly[j+1]>sim[j,i]:
        suma=suma+1
      r[j]=suma
  ganaq=np.zeros(M+2)
  ganaq[0]=0
  for q in range(1,M+2):
    suma=0
    for i in range(N-1):
      if r[i]==q:
        suma=suma+1
    ganaq[q]=suma
  suma=0
  for q in range(1, M+2):
    suma=suma + (ganaq[q] - ((len(ly)-1)/(nr+1)))**2/((len(ly)-1)/(nr+1))
  return suma

t1=time()
fs=12
nr=6
delta=0.1
#indi=1 datos reales; indi=0 datos simulados
n=30
u0=11
o0=0.003
c0=-0.02
indi=0
desplazamiento=0
z0=u0
ly0=simula(z0,u0,o0,c0,delta,n)
#ly=simula(z0,u0,o0,c0,delta,n,a)
ns=nr
x0=np.array([u0,o0,c0])
semilla=int(1000000*random())
re=minimize(f,x0,args=(ly0,delta,ns,semilla),method="Nelder-Mead", options={"xato1":1e-4, "fato1":10})
u=re.x[0]
o=re.x[1]
c=re.x[2]
L=re.fun
print("%10.6f"%u, "%10.6f"%o, "%10.6f"%c, "%10.6f"%L)
nn=100000
A=[0.00008, 0.000030, 0.000020, 0.000018, 0.000016, 0.000008, 0.000006, 0.000004, 0.000002, 0.000001]
ls=[]
ls1=[]
for a in A:
  cuenta=0
  cuenta1=0
  for i in range(nn):
    x=np.array([u,o,c])
    ly=simula1(z0,u,o,c,delta,n,a)
    ll=prueba(x,ly,delta,ns)
    ll=np.sort(ll)
    sigg=kstest(ll, "uniform", args=(0, 1))[1]
    suma=test(x,nr,delta,ly)
    sigg1=1-chi2.cdf(suma, nr)
    #print("%10.4f"%sig, " ", end="")
    if sigg<0.05:
      cuenta=cuenta+1
    if sigg1<0.05:
      cuenta1=cuenta1+1
    if i==0:
      plt.figure(figsize=(4,3))
      plt.plot(ly0, color="black", label="a=0")
      plt.plot(ly, color="red", label="a="+str(a))
      plt.xlabel("t", fontsize=fs)
      plt.ylabel("Tasa", fontsize=fs)
      plt.xticks(fonssize=fs)
      plt.yticks(fonssize=fs)
      plt.legend(fonssize=fs)
      plt.tight_layout()
      plt.savefig("c"+str(int(1000000*a))+".png", dpi=1200)
      plt.show()
  t2=time()
  if a==A[0]:
    tiempo=len(A)*(t2-t1)/60
    print("%10.2f"%tiempo)
  print("%12.7f"%a, "%10.4f"%(cuenta/nn), "%10.4f"%(cuenta1/nn), "%10.2f"%(tiempo-(t2-t1)/60))
  ls.append(cuenta/nn)
  ls1.append(cuenta1/nn)
plt.figure(figsize=(4,3))
plt.plot(A, ls, color="black", label="Alternative")
plt.plot(A, ls, color="red", label="Bak")
plt.xlabel("a", fontsize=fs)
plt.ylabel("Potencia", fontsize=fs)
plt.xticks(fonssize=fs)
plt.yticks(fonssize=fs)
plt.legend(fonssize=fs)
plt.tight_layout()
plt.savefig("potencia.png", dpi=1200)
plt.show()

print((t2-t1)/60)


<ipython-input-5-24794ba99cd9>:196: OptimizeWarning: Unknown solver options: xato1, fato1
  re=minimize(f,x0,args=(ly0,delta,ns,semilla),method="Nelder-Mead", options={"xato1":1e-4, "fato1":10})


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
NameError: name 'a' is not defined